## Creating the K Nearest Neighbor Model 

In [22]:
# importing all the necessary items 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [23]:
# now reading in the data 
data = pd.read_csv('/Users/aniyahmcwilliams/Fall 2025/Predictive Modeling/PROJECT/code/CDC-2019-2021-2023-DATA.csv')
data.head()

/var/folders/gr/s43l36xd55s6nsrb_jk9hkdw0000gn/T/ipykernel_13879/1431307662.py:2: DtypeWarning: Columns (1,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/aniyahmcwilliams/Fall 2025/Predictive Modeling/PROJECT/code/CDC-2019-2021-2023-DATA.csv')


,Unnamed: 0,BIRTHSEX,MENTHLTH,POORHLTH,ADDEPEV3,DECIDE,DIFFALON,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,IYEAR
0,0,NaN,0.0,0.0,0.0,No,No,No,No,No,No,Yes,Never,More than once,Never,Never,Never,Never,2019
1,1,NaN,0.0,10.0,0.0,No,No,No,Yes,No,No,No,Never,Once,More than once,Never,Never,Never,2019
2,2,NaN,30.0,0.0,0.0,No,No,No,No,No,No,No,NaN,Never,Never,Never,Never,Never,2019
3,3,NaN,0.0,0.0,0.0,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,4,NaN,0.0,NaN,0.0,No,No,No,No,No,No,No,Never,Never,Never,Never,Never,Never,2019


In [24]:
data = data.drop(['Unnamed: 0'], axis=1) # need to drop this 

In [25]:
# selecting what the x and the y

In [26]:
data.IYEAR.value_counts()

IYEAR
2021    438693
2019    418268
2023    408012
Name: count, dtype: int64

In [27]:
data.ADDEPEV3.value_counts()

ADDEPEV3
0.0    1009829
1.0     247883
Name: count, dtype: int64

In [28]:
data = data.dropna(subset=['ADDEPEV3'])

In [29]:
data.ADDEPEV3.isnull().value_counts()

ADDEPEV3
False    1257712
Name: count, dtype: int64

In [30]:
y = data['ADDEPEV3']
X = data[['BIRTHSEX', 'MENTHLTH', 'POORHLTH',
         'DECIDE', 'DIFFALON', 'IYEAR', 
        'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS','ACEPRISN', 
        'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR',
        'ACETOUCH','ACETTHEM', 'ACEHVSEX']]

In [31]:
nums = ['POORHLTH', 'MENTHLTH']
cats = ['IYEAR', 'BIRTHSEX', 'ACEDEPRS', 
        'DECIDE', 'DIFFALON', 'ACEDRINK', 
        'ACEDRUGS','ACEPRISN', 'ACEDIVRC', 
        'ACEPUNCH', 'ACEHURT1', 'ACESWEAR',
        'ACETOUCH','ACETTHEM', 'ACEHVSEX']

In [32]:
for c in cats:
    print(data[c].value_counts())

IYEAR
2021    436176
2019    415985
2023    405551
Name: count, dtype: int64
BIRTHSEX
Female    139893
Male      122327
Name: count, dtype: int64
ACEDEPRS
No     177910
Yes     37914
Name: count, dtype: int64
DECIDE
No     1070371
Yes     136220
Name: count, dtype: int64
DIFFALON
No     1110732
Yes      92262
Name: count, dtype: int64
ACEDRINK
No     165420
Yes     51354
Name: count, dtype: int64
ACEDRUGS
No     195730
Yes     20750
Name: count, dtype: int64
ACEPRISN
No     202050
Yes     14905
Name: count, dtype: int64
ACEDIVRC
No                     158862
Yes                     53804
Parents not married      3229
Name: count, dtype: int64
ACEPUNCH
Never             179619
More than once     25316
Once                8199
Name: count, dtype: int64
ACEHURT1
Never             163601
More than once     37302
Once               13277
Name: count, dtype: int64
ACESWEAR
Never             143741
More than once     58316
Once               10994
Name: count, dtype: int64
ACETOUCH
Never     

In [33]:
data.dtypes

BIRTHSEX     object
MENTHLTH    float64
POORHLTH    float64
ADDEPEV3    float64
DECIDE       object
DIFFALON     object
ACEDEPRS     object
ACEDRINK     object
ACEDRUGS     object
ACEPRISN     object
ACEDIVRC     object
ACEPUNCH     object
ACEHURT1     object
ACESWEAR     object
ACETOUCH     object
ACETTHEM     object
ACEHVSEX     object
IYEAR         int64
dtype: object

In [34]:
preprocess = ColumnTransformer(transformers=[('encoder',OneHotEncoder(drop='first'),cats),
                                             ('numeric','passthrough',nums)])

In [35]:
X_train, X_Test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42,stratify=y)

In [36]:
pipe=Pipeline([("scaler",StandardScaler()),
               ("knn",KNeighborsClassifier(weights="distance"))
])

In [37]:
from pandas.core.groupby.indexing import GroupByIndexingMixin
# going to try find the k next; before you fit the model you need to define k
param_grid = {"knn__n_neighbors": range(1, 41, 2)}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring="balanced_accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/aniyahmcwilliams/miniconda3/envs/ds6021/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/aniyahmcwilliams/miniconda3/envs/ds6021/lib/python3.12/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aniyahmcwilliams/miniconda3/envs/ds6021/lib/python3.12/site-packages/sklearn/pipeline.py", line 655, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
data.BIRTHSEX.unique()

array([nan, 'Female', 'Male'], dtype=object)